In [1]:
!pip install gradio langchain_groq langchain-community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
from huggingface_hub import login
from PyPDF2 import PdfReader
import gradio as gr
import os

from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
# Load env vars (if any) and Hugging Face API token
load_dotenv()
login(os.getenv('HUGGINGFACEHUB_API_TOKEN'))


In [4]:
# STEP 3: Upload and process PDF (can replace with fixed path)
from google.colab import files
uploaded = files.upload()
pdf_paths = list(uploaded.keys())

Saving Policy.pdf to Policy.pdf


In [5]:
# Step 4: Extract text
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

raw_text = get_pdf_text(pdf_paths)

In [6]:
# Step 5: Split text into chunks
def get_text_chunks(raw_text):
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)
    return splitter.split_text(raw_text)

text_chunks = get_text_chunks(raw_text)

In [7]:
# Step 6: Create vector store
def get_vectorstore(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device":"cpu"})
    return FAISS.from_texts(texts=chunks, embedding=embeddings)

vectorstore = get_vectorstore(text_chunks)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [8]:
# Step 7: Setup chatbot with memory
def get_conversation_chain(vstore):
    llm = ChatGroq(
        model_name="llama3-8b-8192",
        temperature=0.7,
        request_timeout=30,
        api_key="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"  # Replace this securely
    )
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    return ConversationalRetrievalChain.from_llm(llm=llm, retriever=vstore.as_retriever(), memory=memory)

conversation = get_conversation_chain(vectorstore)

/tmp/ipython-input-36010370.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [9]:
# Step 8: Define chatbot function
def chat_bot(user_input, history=[]):
    result = conversation({"question": user_input})
    reply = result["answer"]
    history.append((user_input, reply))
    return history, history




In [10]:
# Step 9: Launch Gradio API
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    message = gr.Textbox(label="Ask a question about the policy")
    clear = gr.Button("Clear chat")

    message.submit(chat_bot, [message, chatbot], [chatbot, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True)


/tmp/ipython-input-2006372929.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d0ac62b15db03521b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
